# 데코레이터

데코레이터는 함수를 수정하지 않은 상태에서 추가 기능을 구현할 때 사용한다

데코레이터는 함수를 인자로 받아 다른 함수를 반환하는 "callable"한 객체이다</br>
데코레이터는 @ 기호를 사용하여 함수나 클래스 위에 작성하는 형태로 적용한다

기존 함수를 수정하지 않고도 새로운 동작을 추가할 수 있기 때문에, 코드의 재사용성과 모듈성을 향상시킬 수 있다

https://kukuta.tistory.com/336

## 함수에서 데코레이터 활용

In [182]:
def add_message(f):
    '''함수 앞뒤로 시작/종료 메세지를 추가한다'''
    
    def new_func():
        print('처리를 시작합니다')
        f()                    # sample_func()
        print('처리를 종료합니다')
        
    return new_func

@add_message
def sample_func():
    '''실행 메세지를 표시하는 함수'''
    print('sample_func 함수 처리를 실행합니다')
    
deco_func()

처리를 시작합니다
sample_func 함수 처리를 실행합니다
처리를 종료합니다


In [138]:
def add_message(f):
    '''함수 앞뒤로 시작/종료 메세지를 추가한다'''
    
    def new_func():
        print('처리를 시작합니다')
        f()
        print('처리를 종료합니다')
        
    return new_func

def sample_func():
    '''실행 메세지를 표시하는 함수'''
    print('sample_func 함수 처리를 실행합니다')
    
deco_func = add_message(sample_func) # 데코레이터의 동작을 이와 같이 표현할 수 있다
deco_func()

처리를 시작합니다
sample_func 함수 처리를 실행합니다
처리를 종료합니다


함수에는 데코레이터를 여러 개 지정할 수 있다

```python
@decorator1
@decorator2
def 함수이름():
    코드
```

데코레이터의 동작은

```python
decorator1(decorator2(함수이름))
```

와 같다

In [206]:
def decorator1(func):
    def wrapper():
        print('decorator1')
        func()
    return wrapper
 
def decorator2(func):
    def wrapper():
        print('decorator2')
        func()
    return wrapper
 
# 데코레이터를 여러 개 지정
@decorator1
@decorator2
def hello():
    print('hello')

In [207]:
hello()

decorator1
decorator2
hello


### 인자를 가진 함수 데코레이터

데코레이터에 인자를 전달할 수 있다

아래 코드는 num_times에 3을 전달해서, wrapper함수에서 greet함수를 3(num_times)번 반복 실행한다</br>

In [183]:
def repeat(num_times):
    def decorator_function(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_function

@repeat(num_times=3)
def greet(name):
    print(f"Hello, {name}!")

greet("Alice")

Hello, Alice!
Hello, Alice!
Hello, Alice!


데코레이터의 목적은 다른 함수를 감싸는 것이므로, 데코레이터의 inner 함수는 원본 함수와 동일한 시그니처를 가져야 한다</br>
이를 위해 \*args와 \*\*kwargs 매개변수를 사용하여 유연성을 제공하고, 데코레이터를 적용한 함수의 인자를 적절히 처리할 수 있다

In [271]:
def add_emoticon(func):           # func 매개변수에 say_hi 함수가 입력
    
    def inner(*args, **kwargs):   # 데코레이터에 감싸지는 say_hi함수가 인자를 받기 때문에
                                  # 데코레이터의 inner함수도 인자를 받을 수 있도록 매개변수를 정의해야 한다
                                  # 가변인자를 입력하면, 데코레이터를 적용한 함수가 다양한 인자를 받더라도 INNER함수에서 인자들을 처리할 수 있다
        print('=￣ω￣=', end = '\n')
        func(*args, **kwargs)
    return inner

@add_emoticon
def say_hi(name):
    print(f'hi {name}!')

In [272]:
say_hi('Jason')

=￣ω￣=
hi Jason!


## 클래스에서 데코레이터 활용

```python
def add_emoticon(func):
    
    def inner(*args, **kwargs):
        print('=￣ω￣=', end = '\n')
        func(*args, **kwargs)
    return inner

@add_emoticon
def say_hi(name):
    print(f'hi {name}!')
```

위 코드에서 @add_emoticon 데코레이터를 사용하면 인터프리터에 의해 say_hi = add_emoticon(say_hi)와 같은 코드로 바뀐다</br>
이 형태는 클래스 객체 생성과 동일하다

이를 활용하여 클래스 데코레이터로 변경한다

하지만, 생성자로 add_emoticon(say_hi) 형태로 처리해도 현재 say_hi는 함수가 아니라 클래스 객체이기 때문에 함수처럼 호출 할 수 없다</br>
__call__ 매직 메서드를 사용해서 객체를 함수처럼 호출 할 수 있도록 처리해야 한다

In [258]:
class AddEmoticon:
    def __init__(self, func):             # add_emoticon(say_hi) 처리
        self.func = func
    def __call__(self, *args, **kwargs):  # 실제 데코레이팅
                                          # 객체를 함수처럼 호출 할 수 있도록 __call__ 매직 메서드를 적용
        print('=￣ω￣=', end = '\n')
        return self.func(*args, **kwargs)
        
@AddEmoticon
def say_hi(name):
    print(f'hi {name}!')

In [259]:
say_hi('Jason')                            # say_hi.__call__('Jason')과 동일

=￣ω￣=
hi Jason!


In [260]:
print(type(say_hi))                        # <class '__main__.AddEmoticon'>

<class '__main__.AddEmoticon'>


### 인자를 가진 클래스 데코레이터

데코레이터를 @AddEmoticon(인자) 형태로 지정하면, say_hi = AddEmoticon(인자)(say_hi)와 같은 코드를 인터프리터가 생성한다</br>
이와 같은 형태라면 AddEmoticon 클래스의 생성자는 함수를 넘겨 받는 것이 아니라 이모티콘을 넘겨 받는다

인터프리터는 객체 생성 이후에 바로 함수를 인자로 넘겨주는 호출을 하고 있다</br>
AddEmoticon 클래스가 생성자에서 이모티콘을 받도록 수정 되면서 오히려 함수를 넘겨 받는 부분이 없어졌다

\_\_call\_\_ 메소드가 say_hi함수를 인자로 받고, 데코레이팅된 inner함수를 리턴하도록 하면 된다

In [273]:
class AddEmoticon:
    # def __init__(self, func):
    #    self.func = func
    
    def __init__(self, emoticon):          # 함수 대신 이모티콘을 인자로 받아 저장
        self.emoticon = emoticon
        
    # def __call__(self, *args, **kwargs):
    #    print('^_^ ', end='')
    #    return self.func(*args, **kwargs)
    
    def __call__(self, func):
        
        def inner(*args, **kwargs):
            print(self.emoticon, end='\n')
            return func(*args, **kwargs)
            
        return inner

@AddEmoticon('=￣ω￣=')                      # say_hi = add_emoticon('=￣ω￣=')(say_hi)
def say_hi(name) :
    print(f'hi {name}!')

In [269]:
say_hi('Jason')

=￣ω￣=
hi Jason!


### 상태를 가지는 데코레이터

callable object를 사용하면 함수처럼 사용할 수 있으면서도 상태를 저장할 수 있다

AddEmoticon 데코레이터를 이용한 say_hi 함수가 몇 번 호출되었는지 추적할 수 있도록 위 예제를 변형 해본다

In [288]:
class AddEmoticon:
    
    def __init__(self, emoticon):         
        self.count = 0                       # 호출 회수 저장 필드
        self.emoticon = emoticon
    
    def __call__(self, func):
        
        def inner(*args, **kwargs):
            self.count += 1
            inner.count = self.count         # 함수 객체에서 바로 사용 할 수 있도록
            print(self.emoticon, end='\n')
            return func(*args, **kwargs)
            
        return inner
        
@AddEmoticon('=￣ω￣=')    
def say_hi(name):
    print(f'hi {name}!')

In [289]:
say_hi('Jason')

=￣ω￣=
hi Jason!


객체를 생성할 때마다 inner 함수에서 self.count += 1이 작동되어 count가 늘어난다</br>
\_\_call\_\_ 메서드가 inner를 return해서 say_hi객체(함수)로 반환하기 때문에</br>
inner.count = self.count 형태로 개개체의 인스턴스 변수로 전달해서 say_hi.count 형태로 접근할 수 있도록 했다

In [290]:
say_hi.count

1